# SQL Assignment

In [1]:
import pandas as pd
import sqlite3
import time
from IPython.display import display, HTML

In [2]:
# Note that this is not the same db we have used in course videos, please download from this link
# https://drive.google.com/file/d/1O-1-L1DdNxEK6O6nG2jS31MbrMh-OnXM/view?usp=sharing

In [3]:
conn = sqlite3.connect("Db-IMDB-Assignment.db")

#### Overview of all tables

In [4]:
tables = pd.read_sql_query("SELECT NAME AS 'Table_Name' FROM sqlite_master WHERE type='table'",conn)
tables = tables["Table_Name"].values.tolist()

In [5]:
for table in tables:
    query = "PRAGMA TABLE_INFO({})".format(table)
    schema = pd.read_sql_query(query,conn)
    print("Schema of",table)
    display(schema)
    print("-"*100)
    print("\n")

Schema of Movie


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,title,TEXT,0,None,0
3,3,year,TEXT,0,None,0
4,4,rating,REAL,0,None,0
5,5,num_votes,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Genre


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Name,TEXT,0,None,0
2,2,GID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Language


...

----------------------------------------------------------------------------------------------------


Schema of M_Director


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of M_Cast


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------




## Useful tips:

1. the year column in 'Movie' table, will have few chracters other than numbers which you need to be preprocessed, you need to get a substring of last 4 characters, its better if you convert it as int type, ex: CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)

2. For almost all the TEXT columns we have show, please try to remove trailing spaces, you need to use TRIM() function

3. When you are doing count(coulmn) it won't consider the "NULL" values, you might need to explore other alternatives like Count(*)

## Q1 --- List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

<h4>To determine whether a year is a leap year, follow these steps:</h4>

<ul>
    <li><b>STEP-1:</b> If the year is evenly divisible by 4, go to step 2. Otherwise, go to step 5.</li>
    <li><b>STEP-2:</b> If the year is evenly divisible by 100, go to step 3. Otherwise, go to step 4.</li>
    <li><b>STEP-3:</b> If the year is evenly divisible by 400, go to step 4. Otherwise, go to step 5.</li>
    <li><b>STEP-4:</b> The year is a leap year (it has 366 days).</li>
    <li><b>STEP-5:</b> The year is not a leap year (it has 365 days).</li>
</ul>

Year 1900 is divisible by 4 and 100 but it is not divisible by 400, so it is not a leap year.

In [6]:
%%time
def grader_1(q1):
    q1_results  = pd.read_sql_query(q1,conn)
    print(q1_results.head(10))
    assert (q1_results.shape == (232,3))

query1 = """SELECT P.Name as director_name  , m.title as movie_name ,m.year as mod_year from Movie m \
                    INNER JOIN  M_director MD ON MD.MID = m.MID\
                    INNER JOIN Person P   ON  P.PID = MD.PID \
                    INNER JOIN M_Genre mg ON mg.MID = m.MID\
                    INNER JOIN Genre g ON g.GID = mg.GID \
                  WHERE g.Name LIKE '%comedy%' \
                  AND  CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER) % 4 = 0 \
                  or CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER) % 100 <> 0 \
                  and CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)% 400 = 0 """
grader_1(query1)

       director_name                         movie_name mod_year
0       Milap Zaveri                         Mastizaade     2016
1       Danny Leiner  Harold & Kumar Go to White Castle     2004
2     Anurag Kashyap                 Gangs of Wasseypur     2012
3       Frank Coraci        Around the World in 80 Days     2004
4      Griffin Dunne             The Accidental Husband     2008
5        Anurag Basu                             Barfi!     2012
6    Gurinder Chadha                  Bride & Prejudice     2004
7         Mike Judge    Beavis and Butt-Head Do America     1996
8   Tarun Mansukhani                            Dostana     2008
9       Shakun Batra                      Kapoor & Sons     2016
Wall time: 191 ms


## Q2 --- List the names of all the actors who played in the movie 'Anand' (1971)

In [7]:
%%time
def grader_2(q2):
    q2_results  = pd.read_sql_query(q2,conn)
    print(q2_results.head(10))
    assert (q2_results.shape == (17,1))


query2 = """select p.Name  as Actor_Names from Person p \
            JOIN M_cast ON TRIM(p.PID) = TRIM(M_cast.PID) \
            WHERE MID IN(select MID from Movie M WHERE title ='Anand') """
grader_2(query2)

         Actor_Names
0      Rajesh Khanna
1   Amitabh Bachchan
2      Sumita Sanyal
3         Ramesh Deo
4          Seema Deo
5     Asit Kumar Sen
6         Dev Kishan
7       Atam Prakash
8      Lalita Kumari
9             Savita
Wall time: 272 ms


## Q3 --- List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [8]:
%%time

def grader_3a(query_less_1970, query_more_1990):
    q3_a = pd.read_sql_query(query_less_1970,conn)
    q3_b = pd.read_sql_query(query_more_1990,conn)
    return (q3_a.shape == (4942,1)) and (q3_b.shape == (62570,1))

query_less_1970 =""" select p.PID  from person p\
                    JOIN (select TRIM(c.PID) PID , c.MID from M_cast c
                    WHERE c.MID IN(select m.MID from Movie m  WHERE  CAST(SUBSTR(m.year,-4) AS Integer)<1970)) q1 \
                    ON q1.PID = p.PID """

query_more_1990 =""" select p.PID  from person p\
                    JOIN (select TRIM(c.PID) PID , c.MID from M_cast c
                    WHERE c.MID IN(select m.MID from Movie m  WHERE  CAST(SUBSTR(m.year,-4) AS Integer)>1990)) q2 \
                    ON q2.PID = p.PID """
print(grader_3a(query_less_1970, query_more_1990))

# using the above two queries, you can find the answer to the given question 

True
Wall time: 442 ms


In [9]:
%%time
def grader_3(q3):
    q3_results  = pd.read_sql_query(q3,conn)
    print(q3_results.head(10))
    assert (q3_results.shape == (300,1))

query3 = """select DISTINCT p.Name Actors from person p
                    where  PID IN(
                    select DISTINCT TRIM(c.PID)  from M_cast c
                    where c.MID IN(select m.MID from Movie m  WHERE   CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)<1970)
                    INTERSECT
                    select DISTINCT TRIM(c.PID) from M_cast c 
                    where c.MID IN 
                     (select m.MID from Movie m  WHERE   CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)>1990))"""
grader_3(query3)

              Actors
0       Rishi Kapoor
1   Amitabh Bachchan
2             Asrani
3       Zohra Sehgal
4    Parikshat Sahni
5      Rakesh Sharma
6        Sanjay Dutt
7          Ric Young
8              Yusuf
9     Suhasini Mulay
Wall time: 233 ms


## Q4 --- List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [10]:
%%time

def grader_4a(query_4a):
    query_4a = pd.read_sql_query(query_4a,conn)
    print(query_4a.head(10)) 
    return (query_4a.shape == (1462,2))

query_4a ="""select md.pid as Director_id,count(*) movie_Count from movie m, m_director md,person p on trim(m.mid)=trim(md.mid) and \
                     p.pid=md.pid group by md.pid """
print(grader_4a(query_4a))

# using the above query, you can write the answer to the given question

  Director_id  movie_Count
0   nm0000180            1
1   nm0000187            1
2   nm0000229            1
3   nm0000269            1
4   nm0000386            1
5   nm0000487            2
6   nm0000965            1
7   nm0001060            1
8   nm0001162            1
9   nm0001241            1
True
Wall time: 3.02 s


In [11]:
%%time
def grader_4(q4):
    q4_results  = pd.read_sql_query(q4,conn)
    print(q4_results.head(10))
    assert (q4_results.shape == (58,2))

query4 = """select p.name,count(*) Movie_Count from movie m, m_director md,person p on trim(m.mid)=trim(md.mid) and \
                      p.pid=md.pid group by md.pid having count(*)>=10 order by Movie_Count desc"""
grader_4(query4)

                    Name  Movie_Count
0           David Dhawan           39
1           Mahesh Bhatt           35
2        Ram Gopal Varma           30
3           Priyadarshan           30
4           Vikram Bhatt           29
5   Hrishikesh Mukherjee           27
6            Yash Chopra           21
7         Shakti Samanta           19
8        Basu Chatterjee           19
9           Subhash Ghai           18
Wall time: 3.11 s


## Q5.a --- For each year, count the number of movies in that year that had only female actors.

In [12]:
%%time

# note that you don't need TRIM for person table

def grader_5aa(query_5aa):
    query_5aa = pd.read_sql_query(query_5aa,conn)
    print(query_5aa.head(10)) 
    return (query_5aa.shape == (8846,3))

query_5aa =""" select mc.mid,p.gender,count(*) \
                from m_cast mc join person p on p.pid = trim(mc.pid) group by \
                mc.mid,p.gender """

print(grader_5aa(query_5aa))

def grader_5ab(query_5ab):
    query_5ab = pd.read_sql_query(query_5ab,conn)
    print(query_5ab.head(10)) 
    return (query_5ab.shape == (3469, 3))

query_5ab ="""select mc.mid,p.gender,count(*) \
            from m_cast mc join person p on p.pid = trim(mc.pid) where p.gender = 'Male' group by \
            mc.mid,p.gender  """

print(grader_5ab(query_5ab))


# using the above queries, you can write the answer to the given question

         MID  Gender  count(*)
0  tt0021594    None         1
1  tt0021594  Female         3
2  tt0021594    Male         5
3  tt0026274    None         2
4  tt0026274  Female        11
5  tt0026274    Male         9
6  tt0027256    None         2
7  tt0027256  Female         5
8  tt0027256    Male         8
9  tt0028217  Female         3
True
         MID Gender  count(*)
0  tt0021594   Male         5
1  tt0026274   Male         9
2  tt0027256   Male         8
3  tt0028217   Male         7
4  tt0031580   Male        27
5  tt0033616   Male        46
6  tt0036077   Male        11
7  tt0038491   Male         7
8  tt0039654   Male         6
9  tt0040067   Male        10
True
Wall time: 545 ms


In [13]:
%%time
def grader_5a(q5a):
    q5a_results  = pd.read_sql_query(q5a,conn)
    print(q5a_results.head(10))
    assert (q5a_results.shape == (4,2))

query5a = """select CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER) as year,count(distinct m.MID) Female_cast_Movies \
        from Movie m ,M_Cast mc \
        where  m.MID not in
        (select distinct M_Cast.MID from \
         M_Cast  INNER JOIN  Person  \
        on trim(M_Cast.PID)= Person.PID \
        where Person.Gender='Male' 
        )
        and m.MID=mc.MID and mc.PID IS NOT NULL
        group by year
"""
grader_5a(query5a)

   year  Female_cast_Movies
0  1939                   1
1  1999                   1
2  2000                   1
3  2018                   1
Wall time: 377 ms


## Q5.b --- Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [14]:
%%time
def grader_5b(q5b):
    q5b_results  = pd.read_sql_query(q5b,conn)
    print(q5b_results.head(10))
    assert (q5b_results.shape == (4,3))

query5b = """select CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER) as year,(count(distinct m.MID)*100.00)/count(m.MID)\
                as Percentage_Female_Only_Movies ,count(m.MID) Total_movies \
                from Movie m ,M_Cast mc \
        where  m.MID not in\
        (select distinct M_Cast.MID from \
         M_Cast  INNER JOIN  Person  \
        on trim(M_Cast.PID)= Person.PID \
        where Person.Gender='Male' )\
        and m.MID=mc.MID and mc.PID IS NOT NULL\
        group by year """
grader_5b(query5b)

   year  Percentage_Female_Only_Movies  Total_movies
0  1939                     100.000000             1
1  1999                       9.090909            11
2  2000                      10.000000            10
3  2018                      50.000000             2
Wall time: 385 ms


## Q6 --- Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [15]:
%%time
def grader_6(q6):
    q6_results  = pd.read_sql_query(q6,conn)
    print(q6_results.head(10))
    assert (q6_results.shape == (3473, 2))

query6 = """SELECT m.title AS title,  COUNT(DISTINCT PID) as count FROM Movie m, M_Cast mc on  m.MID = mc.MID \
            group by mc.MID order by count desc """
grader_6(query6)

                        title  count
0               Ocean's Eight    238
1                    Apaharan    233
2                        Gold    215
3             My Name Is Khan    213
4  Captain America: Civil War    191
5                    Geostorm    170
6                     Striker    165
7                        2012    154
8                      Pixels    144
9       Yamla Pagla Deewana 2    140
Wall time: 315 ms


### Q7 --- A decade is a sequence of 10 consecutive years. 
### For example, say in your database you have movie information starting from 1931. 
### the first decade is 1931, 1932, ..., 1940,
### the second decade is 1932, 1933, ..., 1941 and so on. 
### Find the decade D with the largest number of films and the total number of films in D.

In [16]:
%%time
def grader_7a(q7a):
    q7a_results  = pd.read_sql_query(q7a,conn)
    print(q7a_results.head(10))
    assert (q7a_results.shape == (78, 2))

query7a = """ select CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER) Movie_year, count(mid) Total_movies from movie m \
                group by Movie_year """
grader_7a(query7a)

# using the above query, you can write the answer to the given question

   Movie_year  Total_movies
0        1931             1
1        1936             3
2        1939             2
3        1941             1
4        1943             1
5        1946             2
6        1947             2
7        1948             3
8        1949             3
9        1950             2
Wall time: 8.82 ms


In [17]:
%%time
def grader_7b(q7b):
    q7b_results  = pd.read_sql_query(q7b,conn)
    print(q7b_results.head(10))
    assert (q7b_results.shape == (713, 4))

query7b = """   
               select tabel1.Movie_year Movie_year,tabel1.Total_movies Total_movies,\
                    tabel2.Movie_year Movie_year, tabel2.Total_movies Total_movies from\
                  (select CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER) Movie_year, count(mid) Total_movies from movie m\
                  group by Movie_year) tabel1  join \
                (select CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER) Movie_year, count(mid) Total_movies from movie m\
                  group by Movie_year) tabel2\
                  where tabel1.Movie_year <= tabel2.Movie_year and \
                  tabel2.Movie_year <= tabel1.Movie_year+9 """
grader_7b(query7b)
# if you see the below results the first movie year is less than 2nd movie year and 
# 2nd movie year is less or equal to the first movie year+9

# using the above query, you can write the answer to the given question

   Movie_year  Total_movies  Movie_year  Total_movies
0        1931             1        1931             1
1        1931             1        1936             3
2        1931             1        1939             2
3        1936             3        1936             3
4        1936             3        1939             2
5        1936             3        1941             1
6        1936             3        1943             1
7        1939             2        1939             2
8        1939             2        1941             1
9        1939             2        1943             1
Wall time: 20 ms


In [18]:
%%time
def grader_7(q7):
    q7_results  = pd.read_sql_query(q7,conn)
    print(q7_results.head(10))
    assert (q7_results.shape == (1, 2))

query7 = """select  count(*) as Decade_Movie_count , m1.year as Decade \

            from (select distinct year from movie) m1 join \
            movie m \
            on  CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)>= m1.year \
            and CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER) < (m1.year + 10) \
            group by m1.year \
            order by count(*) desc \
            limit 1"""
grader_7(query7)
# if you check the output we are printinng all the year in that decade, its fine you can print 2008 or 2008-2017

   Decade_Movie_count Decade
0                1203   2008
Wall time: 489 ms


## Q8 --- Find all the actors that made more movies with Yash Chopra than any other director.

In [19]:
%%time
def grader_8a(q8a):
    q8a_results  = pd.read_sql_query(q8a,conn)
    print(q8a_results.head(10))
    assert (q8a_results.shape == (73408, 3))

query8a = """select  p.PID as actor, \
                        md.PID as director,\
                        count(*) movies \
                        from M_director md join \
                        person p \
                        on p.pid = trim(md.pid) join M_cast mc \
                        on mc.mid = md.mid join person pa\
                        on pa.PID = trim(mc.pid) \
                        group by pa.PID,md.PID"""
grader_8a(query8a)

# using the above query, you can write the answer to the given question

       actor   director  movies
0  nm0496746  nm0496746       1
1  nm0000180  nm0000180       1
2  nm0896533  nm0896533       1
3  nm0896533  nm0896533       1
4  nm0004292  nm0004292       1
5  nm0485943  nm0485943       1
6  nm0000229  nm0000229       1
7  nm0178997  nm0178997       1
8  nm0000269  nm0000269       1
9  nm0113819  nm0113819       1
Wall time: 800 ms


In [20]:
%%time

def grader_8(q8):
    q8_results  = pd.read_sql_query(q8,conn)
    print(q8_results.head(10))
    print(q8_results.shape)
    assert (q8_results.shape == (245, 2))

query8 = """ select mda.actor, mda.movies_with_chopra from \
            (SELECT  pd.pid,pd.name as Director, pa.pid, pa.name as actor,\
               count(*) as movies_with_chopra, \
               RANK() OVER (PARTITION BY pa.pid ORDER BY COUNT(*) DESC) as seqnum, \
                COUNT(*) OVER (PARTITION BY pa.pid, COUNT(*)) as num_with_cnt \

               from M_director md join person pd \
               on pd.pid = trim(md.pid)  join \
               M_cast mc on mc.mid = md.mid join \
               person pa on pa.pid = trim(mc.pid) \
               group by pd.pid, pd.name,pa.pid,pa.name) mda \
               WHERE Director LIKE '%Yash Chopra%' and seqnum = 1 order by movies_with_chopra desc"""
grader_8(query8)

               actor  movies_with_chopra
0        Jagdish Raj                  11
1   Manmohan Krishna                  10
2           Iftekhar                   9
3      Shashi Kapoor                   7
4      Rakhee Gulzar                   5
5     Waheeda Rehman                   5
6           Ravikant                   4
7     Achala Sachdev                   4
8        Neetu Singh                   4
9      Leela Chitnis                   3
(245, 2)
Wall time: 1.34 s


## Q9 --- The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [21]:
%%time
def grader_9a(q9a):
    q9a_results  = pd.read_sql_query(q9a,conn)
    print(q9a_results.head(10))
    print(q9a_results.shape)
    assert (q9a_results.shape == (2382, 1))

query9a = """ with q1 as (select pid from person p where trim(p.name) = 'Shah Rukh Khan'),\
                  q2 as (select mc.mid from M_cast mc join q1 \
                where trim(mc.pid) = q1.pid), \
                q3 as (select p1.pid from person p1 where p1.pid in (select trim(mc.pid) from m_cast mc join q2 on\
                mc.mid=q2.mid) and p1.pid!='nm0451321' ) \
                select * from q3 """
grader_9a(query9a)
# using the above query, you can write the answer to the given question

# selecting actors who acted with srk (S1)
# selecting all movies where S1 actors acted, this forms S2 movies list
# selecting all actors who acted in S2 movies, this gives us S2 actors along with S1 actors
# removing S1 actors from the combined list of S1 & S2 actors, so that we get only S2 actors

         pid
0  nm5951787
1  nm7247557
2  nm7255636
3  nm7737551
4  nm4771952
5  nm3037642
6  nm0007102
7  nm1817397
8  nm1664541
9  nm3385526
(2382, 1)
Wall time: 4.68 s


In [22]:
%%time
def grader_9(q9):
    q9_results  = pd.read_sql_query(q9,conn)
    print(q9_results.head(10))
    print(q9_results.shape)
    assert (q9_results.shape == (25698, 1))

query9 = """select name Actor_name from person where pid in \
                    (select pid from person where pid in \
                    (select trim(pid) from m_cast where mid in \
                    (select mid from m_cast where trim(pid) in \
                    (select p.pid from person p join m_cast mc on trim(mc.pid)=p.pid and \
                    trim(p.name)!='Shah Rukh Khan' and mc.mid in \
                 (select mid from m_cast where trim(pid) in \
                 (select pid from person p where trim(p.name) ='Shah Rukh Khan') ) ) )\
                 and  pid not in \
                    (select p.pid from person p join m_cast mc on trim(mc.pid)=p.pid and \
                    trim(p.name)!='Shah Rukh Khan' and mc.mid in \
                    (select mid from m_cast where pid in \
                    (select pid from person p where trim(p.name) ='Shah Rukh Khan' ) ) )))\
                    and pid not in \
                    (select pid from person where pid in \
                    (select trim(pid) from m_cast where mid in \
                    (select mid from m_cast where trim(pid) in \
                    (select p.pid from person p join m_cast mc on trim(mc.pid)=p.pid and trim(p.name)='Shah Rukh Khan'\
                    and mc.mid in \
                    (select mid from m_cast where trim(pid) in \
                    (select pid from person p where trim(p.name) ='Shah Rukh Khan') ) ))\
                  and pid not in \
                  (select p.pid from person p join m_cast mc on trim(mc.pid)=p.pid and trim(p.name)!='Shah Rukh Khan' and mc.mid in \
                (select mid from m_cast where pid in \
               (select pid from person p where trim(p.name) ='Shah Rukh Khan')))))"""
grader_9(query9)

               Actor_name
0            Freida Pinto
1             Rohan Chand
2            Damian Young
3         Waris Ahluwalia
4   Caroline Christl Long
5           Rajeev Pahuja
6       Michelle Santiago
7         Alicia Vikander
8            Dominic West
9          Walton Goggins
(25698, 1)
Wall time: 901 ms
